# Example 4: Ternary Tangent Pinch

#### Example 3 depicted a tangent pinch for a binary system.  This example shows how a tangent pinch is represented in a ternary system of Acetaldehyde, Methanol, and Water.

In these systems, as Reflux Ratio changes, the fixed point which terminates the rectifying section changes.  However, at the tangent pinch, a very small increment in reflux ratio causes the fixed point composition to rapidly shoot to a new value. 

This example shows the rectifying section experiences a jump at this tangent pinch, which occurs at a reflux ratio between 0.1 and 0.2.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from src.thermo_models.RaoultsLawModel import *
from src.utils.AntoineEquation import * 
from IPython.display import clear_output

from src.distillation.DistillationTernary import *
import ipywidgets as widgets
from IPython.display import display

from ipywidgets import interactive, Button, VBox, HBox, Output
from IPython.display import clear_output
from src.thermo_models.MargulesModel import *
from src.distillation.residue_curves import *

/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
/Applications/Anaconda/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


###### System parameters using Margules Model

In [2]:
A_ = np.array([[0, -316.699, 350.100], [-384.657, 0, 307.000],[290.200, 143.00, 0]])

#Example 4 in Fidkowski is at 1 atm = 760 mmHg
P_sys = 1.01325 #bar

#Antoine parameters from table 2.1 in page 25 of Doherty Malone Distillation Textbook
#Units: P_sat in mmHg and Temperature in Celsius 
#Form is P_sat = A - B/(T+C)

#Antoine parameters from NIST (Kelvin, Bar)
#Acetaldehyde
Acet_A = 3.68639
Acet_B = 822.894
Acet_C = -69.899
#Methanol
Me_A = 5.31301
Me_B = 1676.569
Me_C = -21.728
#Water
H2O_A = 3.55959
H2O_B = 643.748
H2O_C = -198.043

#Antoine Equations 
Acet_antoine     = AntoineEquationBase10(Acet_A, Acet_B, Acet_C)
H2O_antoine      = AntoineEquationBase10(H2O_A, H2O_B, H2O_C)
Methanol_antoine = AntoineEquationBase10(Me_A, Me_B, Me_C)

In [3]:
# Create a VLE model object
vle_model = MargulesModel(3, P_sys, A_,["Acet", "Meth","H2O"],[Acet_antoine, Methanol_antoine, H2O_antoine], True)

# Create a distillation model
distillation_model = DistillationModelTernary(vle_model, xF = np.array([0.3, 0.3, 0.4]), xD = np.array([0.5030, 0.4960, 0.0010]), xB = np.array([1e-10, 0.0103 - 1e-10,0.9897]), reflux = 0.1)

# Create a residue curve map object
rcm = PhasePortraits(vle_model, distillation_model)

In [4]:
def plot_rect(dist_model, rcm, rr):
    
    dist_model.set_r(rr)    
    fig, ax = plt.subplots(1,2,figsize= (10,5))
    dist_model.plot_rect_comp(ax[0])
    
    rcm.plot_vector_field_rect (ax[1], 20)
    dist_model.plot_rect_comp(ax[1])
    
    plt.show()

def plot_rect_strip(dist_model, rcm, rr):
    
    dist_model.set_r(rr)
    
    fig, ax = plt.subplots(1,3,figsize= (15,5))
    dist_model.plot_rect_strip_comp(ax[0])   
    
    rcm.plot_vector_field_rect  (ax[1], 20)
    rcm.plot_vector_field_strip (ax[2], 20)
    
    dist_model.plot_rect_comp  (ax[1])
    dist_model.plot_strip_comp (ax[2])

    plt.show()

In [5]:
# Setup sliders

R1_slider = widgets.FloatSlider(value=0.1, min=0, max=1, step=0.01,  description='Reflux Ratio:', readout_format='.2f')
R2_slider = widgets.FloatSlider(value=0.1, min=0, max=1, step=0.01,  description='Reflux Ratio:', readout_format='.2f')
go_button = widgets.Button(description='Create Model', width='15%', margin='0 10px 0 0')

# Setup interactive objects

out_1 = Output()
out_2 = Output()

### Demo 1: Rectifying Section

Use the slider to vary the reflux ratio.  Notice how the end of the rectifying section always changes with the reflux ratio, but experiences a much more rapid change between values of 0.1 and 0.2.

In [6]:
def on_go_button_click(b):
    with out_1:
        clear_output(wait=True)
        plt.close('all')  # Close all open matplotlib plots
        plot_rect(distillation_model, rcm, rr = R1_slider.value)
        
go_button = Button(description='Generate Plot')
go_button.on_click(on_go_button_click)

layout = VBox([HBox([R1_slider, go_button]), out_1])
display(layout)

### Demo 2: Full Column

What are the implications of this ternary tangent pinch?

As Example 2 discussed, for a ternary column to be feasible, the rectifying and stripping sections need to overlap.
Change the reflux ratio again, and notice how the jump at the ternary pinch value is necessary for the column to be feasible.  This demonstration also highlights how slight changes to reflux ratio have a much larger impact on the rectifying section than on the stripping section.

In [7]:

def on_go2_button_click(b):
    with out_2:
        clear_output(wait=True)
        plt.close('all')  # Close all open matplotlib plots
        plot_rect_strip( distillation_model, rcm, rr = R2_slider.value)
        
go2_button = widgets.Button(description='Generate Plot')
go2_button.on_click(on_go2_button_click)

layout = VBox([HBox([R2_slider, go2_button]), out_2])
display(layout)